<a href="https://colab.research.google.com/github/kalaiyarasan105/heath_chain/blob/main/Library_book_lending_blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import hashlib
import time
import json


class Block:
    def __init__(self, index, timestamp, transactions, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_string = json.dumps({
            'index': self.index,
            'timestamp': self.timestamp,
            'transactions': self.transactions,
            'previous_hash': self.previous_hash,
            'nonce': self.nonce
        }, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    def mine_block(self, difficulty):
        while self.hash[:difficulty] != "0" * difficulty:
            self.nonce += 1
            self.hash = self.calculate_hash()
        print(f"✅ Block mined: {self.hash}")


class LibraryLendingBlockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.difficulty = 3
        self.pending_transactions = []
        self.authorized_librarians = {
            "L001": "Central Library",
            "L002": "Community Library",
            "L003": "University Library"
        }

    def create_genesis_block(self):
        return Block(0, time.time(), ["Genesis Block: Library Lending Records"], "0")

    def get_latest_block(self):
        return self.chain[-1]

    def mine_pending_transactions(self, librarian_id):
        if librarian_id not in self.authorized_librarians:
            print("⛔ Unauthorized Librarian ID. Cannot mine transactions.")
            self.pending_transactions = []
            return

        block = Block(
            len(self.chain),
            time.time(),
            self.pending_transactions,
            self.get_latest_block().hash
        )
        block.mine_block(self.difficulty)

        self.chain.append(block)
        print(f"📚 Block validated by {self.authorized_librarians[librarian_id]}")
        self.pending_transactions = []

    def add_book_record(self, librarian_id, book_title, patron_name, action):
        if librarian_id not in self.authorized_librarians:
            print("⛔ Unauthorized librarian! Only registered librarians can add records.")
            return

        librarian_name = self.authorized_librarians[librarian_id]
        self.pending_transactions.append({
            "book_title": book_title,
            "patron_name": patron_name,
            "action": action,
            "librarian": librarian_name
        })
        print(f"✅ Record added by {librarian_name}")

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]

            if current_block.hash != current_block.calculate_hash():
                return False
            if current_block.previous_hash != previous_block.hash:
                return False
        return True

    def print_chain(self):
        for block in self.chain:
            print(json.dumps({
                "index": block.index,
                "timestamp": block.timestamp,
                "transactions": block.transactions,
                "hash": block.hash,
                "previous_hash": block.previous_hash
            }, indent=4))
            print("-" * 40)

    def verify_book_status(self, book_title):
        found_records = []
        for block in self.chain:
            for tx in block.transactions:
                if isinstance(tx, dict) and tx.get("book_title") == book_title:
                    found_records.append(tx)

        if found_records:
            print(f"\n🔍 History for {book_title}:")
            for rec in found_records:
                print(json.dumps(rec, indent=4))
        else:
            print(f"\n⚠️ No records found for {book_title}.")


# --- CLI Demo ---
if __name__ == "__main__":
    library_chain = LibraryLendingBlockchain()

    while True:
        print("\n=== Library Lending Blockchain with Librarian Authorization ===")
        print("1. Add book record (Librarian only)")
        print("2. Mine block (validate records)")
        print("3. Show blockchain")
        print("4. Validate blockchain")
        print("5. Verify book status by title (Patron/Librarian Access)")
        print("6. Exit")

        choice = input("Choose option: ")

        if choice == "1":
            lib_id = input("Enter Librarian ID: ")
            title = input("Book Title: ")
            patron = input("Patron Name: ")
            action = input("Action (Borrow/Return): ")
            library_chain.add_book_record(lib_id, title, patron, action)

        elif choice == "2":
            lib_id = input("Enter Librarian ID for validation: ")
            library_chain.mine_pending_transactions(lib_id)

        elif choice == "3":
            library_chain.print_chain()

        elif choice == "4":
            print("Blockchain valid?", library_chain.is_chain_valid())

        elif choice == "5":
            title = input("Enter Book Title to verify: ")
            library_chain.verify_book_status(title)

        elif choice == "6":
            break

        else:
            print("Invalid choice, try again.")


=== Library Lending Blockchain with Librarian Authorization ===
1. Add book record (Librarian only)
2. Mine block (validate records)
3. Show blockchain
4. Validate blockchain
5. Verify book status by title (Patron/Librarian Access)
6. Exit
Choose option: 1
Enter Librarian ID: L001
Book Title: The Hitchhiker's Guide to the Galaxy
Patron Name: Arthur Dent
Action (Borrow/Return): Borrow
✅ Record added by Central Library

=== Library Lending Blockchain with Librarian Authorization ===
1. Add book record (Librarian only)
2. Mine block (validate records)
3. Show blockchain
4. Validate blockchain
5. Verify book status by title (Patron/Librarian Access)
6. Exit
Choose option: 1
Enter Librarian ID: L001
Book Title: The Hitchhiker's Guide to the Galaxy
Patron Name: Arthur Dent
Action (Borrow/Return): Return
✅ Record added by Central Library

=== Library Lending Blockchain with Librarian Authorization ===
1. Add book record (Librarian only)
2. Mine block (validate records)
3. Show blockchain
4. 